In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch
!pip install numpy
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [4]:
# GPU 사용 여부 체크
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

print(DEVICE)

cuda


In [41]:
batch_size = 32
epochs = 5

In [42]:
path = '/content/drive/MyDrive/data/ComputerVision'

In [43]:
# MNIST Train 데이터 받아오기
train_dataset = datasets.MNIST(root=path, train = True, download = True, transform = transforms.ToTensor())
# MNIST Test 데이터 받아오기
test_dataset = datasets.MNIST(root=path, train = False, download = True, transform = transforms.ToTensor())

In [44]:
# 받아온 MNIST데이터를 batch_size로 분리해서 변환
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle=True)

In [45]:
for x, y in train_loader:
  print('변환 후 데이터 Shape : ', x.size())
  print('레이블 데이터 수 : ', y.size())
  break

변환 후 데이터 Shape :  torch.Size([32, 1, 28, 28])
레이블 데이터 수 :  torch.Size([32])


In [46]:
# CNN 모델 정의
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Convolution Layer 
    self.conv1 = nn.Conv2d(
        in_channels = 1,
        out_channels = 8,
        kernel_size = 3,
        padding = 1
    )

    # Convolution Layer 
    self.conv2 = nn.Conv2d(
        in_channels = 8,
        out_channels = 16,
        kernel_size = 3,
        padding = 1
    )

    # Pooling Layer
    self.pool = nn.MaxPool2d(
        kernel_size = 2
    )

    # MLP 적용 후 Classification(0 ~ 9(10개)중 출력)
    self.fc1 = nn.Linear(28 * 28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 10)
    

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    # Flatten
    x = x.view(-1, 28 * 28)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.log_softmax(x)
    return x

In [47]:
model = CNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
)


In [48]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [49]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [50]:
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.300843
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.411210
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.110800
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.305054
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.078446
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.170565
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.236088
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.138760
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.212492
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.010111

[EPOCH: 1], 	Test Loss: 0.0024, 	Test Accuracy: 97.56 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.013687
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.151592
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.098539
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.074938
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.012479
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.189503
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.333624
T